In [1]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

# import random
# text_df = pd.read_csv('./baseline/train.csv')
# random.seed(0)
# indices = list(text_df.index.values)
# random.shuffle(indices)
# text_df = text_df.loc[indices].reset_index(drop=True)
# text_df.to_csv('./train_shuffle.csv', index=False)

In [2]:
special_df = pd.read_csv('./special_to_normal_first.csv')
text_df = pd.read_csv('./train_shuffle.csv')

In [3]:
# 얘네는 변경이 아니라 삭제할 특수문자들
special_df[special_df['convert'].isna()]

,specials,convert
13,',NaN
48,′,NaN
51,±,NaN
52,“,NaN
53,”,NaN
54,″,NaN
84,`,NaN
91,˝,NaN
92,´,NaN
98,#,NaN


In [4]:
# 보류로 작성된 특수문자들
undetermined = special_df.loc[special_df['convert']=='보류', 'specials'].values
undetermined_df = pd.DataFrame(undetermined, columns=['specials'])
undetermined, len(undetermined)

(array(['－', '-', '&', '+', 'δ', '×', '=', '〈', '」', '「', '―', '／', '≤',
        'ㅇ', '*', '≥', '＞', '＜', '÷', '〉', '＼', '』', '『', '＆', '≠', 'ν',
        'κ', '⊃', '⊂', '?', '┘', '┗', '┳', '∃', '╋', '┛', '†', 'ㅡ', 'ㅓ',
        'ㅀ', 'ㅣ', '＝', 'ㅜ', '┌', 'ρ', 'ㅗ', '：', 'ㅒ', '▼', '♀', '♂', '└',
        '｜', '─', '┐', '┓', '┏', '|', '�', 'Σ', '∈', '┣', 'ㅧ', '∠', '￢',
        '┍', '┚', 'Π', '↔', '≫', '≪', '【', '╉'], dtype=object),
 73)

In [5]:
def find_text(df, special):
    # df에서 special을 찾는다
    ls = []
    for i, row in df.iterrows():
        if special in row['invention_title'] or special in row['abstract']:
            ls.append(i)
    return ls

In [6]:
import random

def show_widgets():
    """
    1.현재 special을 보여주는 항목
    
    2.이 special을 바꿀 텍스트를 입력할 항목
    
    3.입력된 텍스트를 저장하고 다음으로 넘어가기 위해 클릭할 버튼

    4.해당 special에 대한 abstract (없으면 ''로 출력)
    5.해당 special에 대한 invention_title (없으면 ''로 출력)
    6.해당 special에 대한 claims (없으면 ''로 출력)

    7.다른 글 보여주기
    
    형식
    [1] [2] [3]
    [7]
    abstract: [4]
    invention_titl: [5]
    claims: [6]
    """
    slider = widgets.IntSlider(min=0, max=len(undetermined)-1, description='slider:')

    def change_idx(change):
        idx = change['new']
        special_box.value = undetermined[idx]
        convert_box.value = ''
        df_indices = find_text(text_df, undetermined[idx])
        abstract_box.value = text_df.loc[df_indices[0], 'abstract']
        invention_title_box.value = text_df.loc[df_indices[0], 'invention_title']
        claims_box.value = text_df.loc[df_indices[0], 'claims'] # claims이 nan인 경우는?

    slider.observe(change_idx, names='value')
    
    special_box = widgets.Text(
        value=f'{undetermined[0]}',
        description='특수문자:',
        disabled=True  
    )
        
    convert_box = widgets.Text(
        value='',
        placeholder='변환할 텍스트 입력',
        description='Convert:',
        disabled=False   
    )

    next_button = widgets.Button(
        description='저장 후 다음으로',
        disabled=False,
    )
    
    def next_click_callback(clicked_button):
        undetermined_df.loc[slider.value, 'convert'] = convert_box.value
        slider.value += 1

    next_button.on_click(next_click_callback)
    
    postpone_button = widgets.Button(
        description='보류',
        disabled=False,
    )

    def postpone_click_callback(clicked_button):
        undetermined_df.loc[slider.value, 'convert'] = '보류'
        slider.value += 1

    postpone_button.on_click(postpone_click_callback)
    
    another_text_button = widgets.Button(
        description='다른 text 보기',
        disabled=False,
    )
    
    def another_text_click_callback(clicked_button):
        # HTML 박스의 내용 바꾸기
        df_indices = find_text(text_df, undetermined[slider.value])
        random.shuffle(df_indices)

        abstract_box.value = text_df.loc[df_indices[0], 'abstract']
        invention_title_box.value = text_df.loc[df_indices[0], 'invention_title']
        claims_box.value = text_df.loc[df_indices[0], 'claims']

    another_text_button.on_click(another_text_click_callback)

    hbox = widgets.HBox([special_box, convert_box, next_button, postpone_button])

    df_indices = find_text(text_df, undetermined[0])

    invention_title_box = widgets.HTML(
        value=text_df.loc[df_indices[0], 'invention_title'],
        description='Title :',
    )
    abstract_box = widgets.HTML(
        value=text_df.loc[df_indices[0], 'abstract'],
        description='Abstract :',
    )
    claims_box = widgets.HTML(
        value=text_df.loc[df_indices[0], 'claims'],
        description='Claims :',
    )

    display(slider, hbox, another_text_button, invention_title_box, abstract_box, claims_box)

show_widgets()

IntSlider(value=0, description='slider:', max=72)

Button(description='다른 text 보기', style=ButtonStyle())

HTML(value='Ｎ－스크린을 통한 콘텐츠-광고 통합 배포 관리 방법 및 시스템', description='Title :')

HTML(value='본 발명은 콘텐츠 프로바이더로부터의 콘텐츠 또는 콘텐츠 관리 시스템에 등록된 콘텐츠와 광고 소재 관리 서버에 등록된 광고 콘텐츠를 동일 형식의 컨테이너에 담아 여러 종류의 이동…

HTML(value='네트워크를 통해 콘텐츠와 광고를 통합하여 사용자 단말로 서비스하는 콘텐츠-광고 통합 배포 관리 시스템에 있어서,콘텐츠 프로바이더(CP)로부터 제공된 파일 형태의 CP 콘텐츠에 …

In [176]:
undetermined_df

,specials
0,－
1,-
2,&
3,+
4,δ
...,...
68,↔
69,≫
70,≪
71,【


In [174]:
undetermined_df.to_csv('./determined.csv', index=False)